In [8]:
import nrrd
import numpy as np
import ndpatch
from glob import glob
import skimage
import os
from scipy import ndimage
import edt
import patchify

In [3]:
def z_padding(input):
    if input.shape[:2] == (256,256):
        pass
    else:
        input=input[:256,:256,:]
    z_size = 128-input.shape[2]%128
    pad = np.zeros(input.shape[0:2]+ (z_size,))
    return np.append(input, pad, axis=-1)

def padding(array, shape):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desirex width
    :return: padded array
    """
    xx, yy, zz = shape
    he, wi, de = array.shape
    print(array.shape)
    a = (xx - he) // 2
    aa = xx - a - he
    print(a, aa)
    b = (yy - wi) // 2
    bb = yy - b - wi
    print(b, bb)

    return np.pad(array, pad_width=((a, aa), (b, bb), (0, 0)), mode='constant')

def gen_train_patches(train_dir, label_dir, out_im, out_la):

    data_list = glob('{}/*.nrrd'.format(train_dir))
    label_list = glob('{}/*.nrrd'.format(label_dir))

    for i, path in enumerate(data_list):
        print('Processing Image ', i+1 ,'/',len(data_list))
        filename = os.path.basename(path).split('.')[0]
        image, header = nrrd.read(path)
        label, head_l = nrrd.read(label_list[i])
        x, y, z = image.shape
        if x<256 and y<256:
            image = padding(image, (256,256,z))
            label = padding(label, (256,256,z))
        else:
            pass
        x, y, z = image.shape
        m_x = x//2
        m_y = y//2
        image = image[m_x-128:m_x+128,m_y-128:m_y+128,:]
        label = label[m_x-128:m_x+128,m_y-128:m_y+128,:]
        if filename[0] == 'D':
            im_win = np.where(image<200,0, image)
            im_win = np.where(image>500,0,im_win)
        else:
            im_win = np.where(image<1100,0, image)
            im_win = np.where(image>1350,0,im_win)
        image_norm = (im_win - np.min(im_win))/(np.max(im_win)-np.min(im_win))
        im_padded = z_padding(image_norm)
        la_padded = z_padding(label)
        #rdm_la = edt.edt(la_padded, black_border=True)
        j=0
        patches_im=patchify.patchify(im_padded, (256,256,128), step=128).squeeze()
        patches_la=patchify.patchify(la_padded, (256,256,128), step=128).squeeze()
        for j in np.arange(patches_im.shape[0]):
            filename_im = out_im + filename + '_' + str(j)
            filename_la = out_la + filename + '_' + str(j)
            if float(np.max(patches_la[j,:,:,:])) == float(0):
                pass
            else:
                nrrd.write(filename_im + '.nrrd', patches_im[j,:,:,:])
                nrrd.write(filename_la + '.seg.nrrd', patches_la[j,:,:,:])

In [14]:
def z_padding(input):
    if input.shape[:2] == (256,256):
        pass
    else:
        input=input[:256,:256,:]
    z_size = 128-input.shape[2]%128
    pad = np.zeros(input.shape[0:2]+ (z_size,))
    return np.append(input, pad, axis=-1)

def padding(array, shape):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desirex width
    :return: padded array
    """
    xx, yy, zz = shape
    he, wi, de = array.shape
    print(array.shape)
    a = (xx - he) // 2
    aa = xx - a - he
    print(a, aa)
    b = (yy - wi) // 2
    bb = yy - b - wi
    print(b, bb)

    return np.pad(array, pad_width=((a, aa), (b, bb), (0, 0)), mode='constant')

def gen_train_patches_test(label_dir, out_la):


    label_list = glob('{}/*.nrrd'.format(label_dir))

    for i, path in enumerate(label_list):
        print('Processing Image ', i+1 ,'/',len(label_list))
        filename = os.path.basename(path).split('.')[0]
        label, head_l = nrrd.read(label_list[i])
        x, y, z = label.shape
        m_x = x//2
        m_y = y//2
        label = label[m_x-128:m_x+128,m_y-128:m_y+128,:]
        la_padded = z_padding(label)
        print(la_padded.shape)
        filename_la = out_la + filename + '_cropped'
        if float(np.max(la_padded)) == float(0):
            pass
        else:
            nrrd.write(filename_la + '.seg.nrrd', la_padded)

In [6]:
train_dir = './training_data/resampled_train/'
label_dir = './training_data/resampled_ground/'
out_im = './training_data/train_z128_iso_win_im/'
out_la = "./training_data/train_z128_iso_win_la/"

gen_train_patches(train_dir, label_dir, out_im, out_la)

Processing Image  1 / 50
Processing Image  2 / 50
Processing Image  3 / 50
Processing Image  4 / 50
Processing Image  5 / 50
Processing Image  6 / 50
Processing Image  7 / 50
Processing Image  8 / 50
Processing Image  9 / 50
Processing Image  10 / 50
Processing Image  11 / 50
Processing Image  12 / 50
Processing Image  13 / 50
Processing Image  14 / 50
Processing Image  15 / 50
Processing Image  16 / 50
Processing Image  17 / 50
Processing Image  18 / 50
Processing Image  19 / 50
Processing Image  20 / 50
Processing Image  21 / 50
Processing Image  22 / 50
Processing Image  23 / 50
Processing Image  24 / 50
Processing Image  25 / 50
Processing Image  26 / 50
(224, 224, 297)
16 16
16 16
(224, 224, 297)
16 16
16 16
Processing Image  27 / 50
Processing Image  28 / 50
Processing Image  29 / 50
Processing Image  30 / 50
Processing Image  31 / 50
Processing Image  32 / 50
Processing Image  33 / 50
Processing Image  34 / 50
Processing Image  35 / 50
Processing Image  36 / 50
Processing Image 

In [61]:
import patchify
import numpy as np
import nrrd
import tensorflow as tf
data, h = nrrd.read('./training/D1.nrrd')


In [62]:
data_z = z_padding(data).astype('int16')
patches = patchify.patchify(data_z, (512,512,48), step=32)
print(patches.shape)
patches = patches.squeeze()
print(patches.shape)

(1, 1, 8, 512, 512, 32)
(8, 512, 512, 32)


In [58]:
nrrd.write('padding_cut.nrrd', data_z)

In [1]:
import numpy as np
import nrrd
data,h =nrrd.read('./train_patch_z32_pad_im/D1_7.nrrd')

In [4]:
image, h = nrrd.read('./test/R1.nrrd')
im_deno = ndimage.median_filter(image, size=2)
image_norm = (im_deno - np.min(im_deno))/(np.max(im_deno)-np.min(im_deno))
img_ad_eq = skimage.exposure.equalize_adapthist(image_norm, clip_limit=0.03)

In [ ]:
import time
start = time.time()
im_frangi = skimage.filters.frangi(img_ad_eq, sigmas=(1,5,10))
end = time.time()
print('frangi filter time: ',(end-start))

In [ ]:
import itk
import time
start = time.time()
image, h = nrrd.read('./test/R1.nrrd')
im_deno = ndimage.median_filter(image, size=2)
image_norm = (im_deno - np.min(im_deno))/(np.max(im_deno)-np.min(im_deno))
img_ad_eq = skimage.exposure.equalize_adapthist(image_norm, clip_limit=0.03)
input = itk.GetImageFromArray(img_ad_eq)
hessian_img = itk.hessian_recursive_gaussian_image_filter(input, sigma=1.0)
vesselness_filter = itk.Hessian3DToVesselnessMeasureImageFilter[itk.ctype("float")].New()
vesselness_filter.SetInput(hessian_img)
vesselness_filter.SetAlpha1(5)
vesselness_filter.SetAlpha2(0.1)
output = itk.GetArrayFromImage(vesselness_filter)
print(np.max(output))
print(np.min(output))

nrrd.write('vessel_test_5_01_s1.nrrd', output)
end = time.time()
print('time: ', end-start)

In [11]:
import time
start = time.time()
vesselness_filter = itk.Hessian3DToVesselnessMeasureImageFilter[itk.ctype("float")].New()
vesselness_filter.SetInput(hessian_img)
vesselness_filter.SetAlpha1(0.5)
vesselness_filter.SetAlpha2(2.0)
output = itk.GetArrayFromImage(vesselness_filter)
print(np.max(output))
print(np.min(output))
nrrd.write('vessel_test_05_2.nrrd', output)
end = time.time()
print('time: ', end-start)


time:  0.0


In [12]:
start = time.time()
vesselness_filter.SetInput(hessian_img)
vesselness_filter.SetAlpha1(0.5)
vesselness_filter.SetAlpha2(2.0)
end = time.time()

In [13]:
output = itk.GetArrayFromImage(vesselness_filter)
print(np.max(output))
print(np.min(output))

909.08215
0.0


In [7]:
train_dir = './training_data/train_z128_iso_im'
label_dir = './training_data/resampled_ground'
data_list = glob('{}/*.nrrd'.format(train_dir))
label_list = glob('{}/*.nrrd'.format(label_dir))

In [9]:
count=0
for i, path in enumerate(data_list):
    image, header = nrrd.read(path)
    print('File: ' ,data_list[i], 'Im dims: ', image.shape)
    print('is nan: ', np.isnan(np.sum(image)) )


File:  ./training_data/train_z128_iso_im\D10_resamp_0.nrrd Im dims:  (256, 256, 128)
is nan:  False
File:  ./training_data/train_z128_iso_im\D10_resamp_1.nrrd Im dims:  (256, 256, 128)
is nan:  False
File:  ./training_data/train_z128_iso_im\D10_resamp_2.nrrd Im dims:  (256, 256, 128)
is nan:  False
File:  ./training_data/train_z128_iso_im\D10_resamp_3.nrrd Im dims:  (256, 256, 128)
is nan:  False
File:  ./training_data/train_z128_iso_im\D11_resamp_0.nrrd Im dims:  (256, 256, 128)
is nan:  False
File:  ./training_data/train_z128_iso_im\D11_resamp_1.nrrd Im dims:  (256, 256, 128)
is nan:  False
File:  ./training_data/train_z128_iso_im\D11_resamp_2.nrrd Im dims:  (256, 256, 128)
is nan:  False
File:  ./training_data/train_z128_iso_im\D12_resamp_0.nrrd Im dims:  (256, 256, 128)
is nan:  False
File:  ./training_data/train_z128_iso_im\D12_resamp_1.nrrd Im dims:  (256, 256, 128)
is nan:  False
File:  ./training_data/train_z128_iso_im\D12_resamp_2.nrrd Im dims:  (256, 256, 128)
is nan:  False


In [16]:
image, h = nrrd.read('./training_data/resampled_train/K19_resamp.nrrd')
label, hl =nrrd.read('./training_data/resampled_ground/K19_resamp.seg.nrrd')

In [17]:
x, y, z = image.shape
print(label.shape)
pad_im = padding(image, (256,256,z))
pad_la = padding(label, (256,256,z))

(224, 224, 297)
(224, 224, 297)
16 16
16 16
(224, 224, 297)
16 16
16 16


In [18]:
print(pad_la.shape)

(256, 256, 297)


In [24]:
nrrd.write('test_pad.nrrd', pad_im)

In [18]:
x,y,z = image.shape
m_x = x//2
m_y = y//2
arr = image[m_x-128:m_x+128,m_y-128:m_y+128,:]
arr_l = label[m_x-128:m_x+128,m_y-128:m_y+128,:]
nrrd.write('test_crop.nrrd',arr)
nrrd.write('test_crop.seg.nrrd', arr_l)

In [22]:
image, h = nrrd.read('training_data/resampled_train/K13_resamp.nrrd')
#img_eq = skimage.exposure.equalize_adapthist(image, clip_limit=0.03)
#im_norm = (image - np.min(image))/(np.max(image)-np.min(image))
#im_norm = np.subtract(im_norm, 0.4, where=(im_norm!=0))
im_win = np.where(image<1100,0, image)
im_win = np.where(image>2000,0,im_win)
nrrd.write('K13_win.nrrd', im_win)